<a href="https://colab.research.google.com/github/opticalcode/Introduction-to-natural-language-processing/blob/master/%E5%A4%AA%E5%AE%B0%E6%B2%BB%E2%80%90%E7%BE%8E%E5%B0%91%E5%A5%B3%E2%80%90%E5%88%86%E3%81%8B%E3%81%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##概要
青空文庫の書籍を分かち書きするスクリプトです。

青空文庫からzip形式でファイルをダウンロードし、分かち書きをして、textファイルでダウンロードができます。

※ MeCabという形態解析ソフトを使って分かち書きを行います。

まずはMeCabのインストールを行います。


In [12]:
!apt-get install mecab
!apt-get install libmecab-dev
!apt-get install mecab-ipadic-utf8
!pip install mecab-python3

Reading package lists... Done
Building dependency tree       
Reading state information... Done
mecab is already the newest version (0.996-5).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libmecab-dev is already the newest version (0.996-5).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
mecab-ipadic-utf8 is already the newest version (2.7.0-20070801+main-1).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


インストールができたので試しに形態解析をしてみます。

In [13]:
import sys
import MeCab
mecab = MeCab.Tagger ("-Owakati")
text = mecab.parse ("すもももももももものうち")
text

'すもも も もも も もも の うち \n'

分かち書きができましたね。

続いて、[青空文庫](https://www.aozora.gr.jp)から書籍をzip形式でダウンロードしてきます。

URLはダウンロードする書籍のURL（.zip）を選択しています。

OUTPUT_FILE は分かち書きが完了した後のファイルの名前を指定します。

In [0]:
URL = 'https://www.aozora.gr.jp/cards/000035/files/242_ruby_20032.zip'
OUTPUT_FILE = 'output.txt'

download( ) は zipファイルをダウンロード、展開、削除、text ファイルを取得します。

In [0]:
import re
import zipfile
import urllib.request
import os.path,glob

def download(url):
  zip_file = re.split(r'/', url)[-1]

  if not os.path.exists(zip_file):
    print('Download URL')
    print('URL:',url)
    urllib.request.urlretrieve(url, zip_file)
  else:
     print('Download File exists')

  dir, ext = os.path.splitext(zip_file)
  if not os.path.exists(dir):
     os.makedirs(dir)

  # open zip
  zip_obj = zipfile.ZipFile(zip_file, 'r')
  zip_obj.extractall(dir)
  zip_obj.close()

  # delete zip
  os.remove(zip_file)

  # get text file path
  path = os.path.join(dir,'*.txt')
  list = glob.glob(path)
  return list[0]

convert() は 青空文庫にはルビが入っていたり、行末に本文とは関係のない文字が含まれているのでそれらを削除します。

※ 文章初めの全角スペースや、改行も一緒に削除しています。

In [0]:
def convert(download_text):
  binarydata = open(download_text, 'rb').read()
  text = binarydata.decode('shift_jis')

  text = re.split(r'\-{5,}', text)[2]
  text = re.split(r'底本：', text)[0]
  text = re.sub(r'《.+?》', '', text)
  text = re.sub(r'［＃.+?］', '', text)
  text = re.sub(r'\u3000', '', text)
  text = re.sub(r'\r\n', '', text)
  text = text.strip()
  return text
 

zipファイルをダウンロードして、文章を取り出します。

In [17]:
download_file = download(URL)

Download URL
URL: https://www.aozora.gr.jp/cards/000035/files/242_ruby_20032.zip


取り出した文章から余計な文字を取り除きます。


In [18]:
text = convert(download_file)
text

'ことしの正月から山梨県、甲府市のまちはずれに小さい家を借り、少しずつ貧しい仕事をすすめてもう、はや半年すぎてしまった。六月にはいると、盆地特有の猛烈の暑熱が、じりじりやって来て、北国育ちの私は、その仮借なき、地の底から湧きかえるような熱気には、仰天した。机の前にだまって坐っていると、急に、しんと世界が暗くなって、たしかに眩暈の徴候である。暑熱のために気が遠くなるなどは、私にとって生れてはじめての経験であった。家内は、からだじゅうのアセモに悩まされていた。甲府市のすぐ近くに、湯村という温泉部落があって、そこのお湯が皮膚病に特効を有する由を聞いたので、家内をして毎日、湯村へ通わせることにした。私たちの借りている家賃六円五拾銭の草庵は、甲府市の西北端、桑畑の中にあり、そこから湯村までは歩いて二十分くらい。（四十九聯隊の練兵場を横断して、まっすぐに行くと、もっと早い。十五分くらいのものかも知れない。）家内は、朝ごはんの後片附がすむと、湯道具持って、毎日そこへ通った。家内の話に依れば、その湯村の大衆浴場は、たいへんのんびりして、浴客も農村のじいさんばあさんたちで、皮膚病に特効があるといっても、皮膚病らしい人は、ひとりも無く、家内のからだが一等きたないくらいで、浴室もタイル張で清潔であるし、お湯のぬるいのが欠点であるけれども、みんな三十分も一時間も、しゃがんでお湯にひたったまま、よもやまの世間話を交して、とにかく別天地であるから、あなたも、一度おいでなさい、ということであった。早朝、練兵場の草原を踏みわけて行くと、草の香も新鮮で、朝露が足をぬらして冷や冷やして、心が豁然とひらけ、ひとりで笑い出したくなるくらいである、という家内の話であった。私は暑熱をいい申しわけにして、仕事を怠けていて、退屈していた時であったから、早速行ってみることにした。朝の八時頃、家内に案内させて、出掛けた。たいしたことも無かった。練兵場の草原を踏みわけて歩いてみても、ひとりで笑い出したくなるようなことは無かった。湯村のその大衆浴場の前庭には、かなり大きい石榴の木が在り、かっと赤い花が、満開であった。甲府には石榴の樹が非常に多い。浴場は、つい最近新築されたものらしく、よごれが無く、純白のタイルが張られて明るく、日光が充満していて、清楚の感じである。湯槽は割に小さく、三坪くらいのものである。浴客が、五人いた

最後にMeCabを使って分かち書きを行います。

In [19]:
text = mecab.parse(text)
text

'ことし の 正月 から 山梨 県 、 甲府 市 の まち はずれ に 小さい 家 を 借り 、 少し ずつ 貧しい 仕事 を すすめ て もう 、 はや 半年 すぎ て しまっ た 。 六月 に は いる と 、 盆地 特有 の 猛烈 の 暑熱 が 、 じりじり やって来 て 、 北国 育ち の 私 は 、 その 仮借 なき 、 地 の 底 から 湧き かえる よう な 熱気 に は 、 仰天 し た 。 机 の 前 に だまっ て 坐っ て いる と 、 急 に 、 しんと 世界 が 暗く なっ て 、 たしかに 眩暈 の 徴候 で ある 。 暑熱 の ため に 気 が 遠く なる など は 、 私 にとって 生れ て はじめて の 経験 で あっ た 。 家内 は 、 からだ じゅう の アセモ に 悩まさ れ て い た 。 甲府 市 の すぐ 近く に 、 湯村 という 温泉 部落 が あっ て 、 そこ の お湯 が 皮膚 病 に 特効 を 有する 由 を 聞い た ので 、 家内 を し て 毎日 、 湯村 へ 通わ せる こと に し た 。 私 たち の 借り て いる 家賃 六 円 五拾 銭 の 草庵 は 、 甲府 市 の 西北 端 、 桑畑 の 中 に あり 、 そこ から 湯村 まで は 歩い て 二 十 分 くらい 。 （ 四 十 九 聯隊 の 練兵 場 を 横断 し て 、 まっすぐ に 行く と 、 もっと 早い 。 十 五 分 くらい の もの かも 知れ ない 。 ） 家内 は 、 朝 ごはん の 後 片 附 が すむ と 、 湯 道具 持っ て 、 毎日 そこ へ 通っ た 。 家内 の 話 に 依れ ば 、 その 湯村 の 大衆 浴場 は 、 たいへん のんびり し て 、 浴客 も 農村 の じいさん ばあさん たち で 、 皮膚 病 に 特効 が ある と いっ て も 、 皮膚 病 らしい 人 は 、 ひとり も 無く 、 家内 の から だ が 一等 きたない くらい で 、 浴室 も タイル 張 で 清潔 で ある し 、 お湯 の ぬるい の が 欠点 で ある けれども 、 みんな 三 十 分 も 一 時間 も 、 しゃがん で お湯 に ひたっ た まま 、 よもやま の 世間 話 を 交し て 、 とにかく 別

これで無事に分かち書きができました！

あとは、先ほど名前をつけた OUTPUT_FILE に保存します。

In [0]:
with open(OUTPUT_FILE,"w") as f:
  f.write(text)

In [21]:
!ls

242_ruby_20032	789_ruby_5639  output.txt  sample_data


output.txt  が保存されていることが確認できたのでダウンロードします。

In [0]:
from google.colab import files
files.download(OUTPUT_FILE)